In [56]:
# !pip install transformers
# !pip install langchain
# !pip install python-dotenv
# !pip install pypdf
# !pip install chromadb
# !pip install sentence-transformers

In [57]:
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] 

In [58]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_community.llms import HuggingFaceEndpoint
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.embeddings import (
    HuggingFaceEmbeddings,
    HuggingFaceBgeEmbeddings,
)
from langchain_community.document_loaders import TextLoader
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub
from dotenv import load_dotenv

In [59]:
## txt
loader = TextLoader("C:\\Users\\shOh\\Downloads\\capstone\\langchain\\test.txt", encoding='utf8')
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)

texts = text_splitter.split_text(data[0].page_content)
pages = loader.load_and_split(text_splitter)

##pdf
# loader = PyPDFLoader("real_data_ex.pdf")

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 500,
#     chunk_overlap  = 100,
#     length_function = len,
# )

# pages = loader.load_and_split(text_splitter)


In [60]:
directory = 'chroma_store'
vector_index = Chroma.from_documents(
    pages, # Documents
    HuggingFaceEmbeddings(), # Text embedding model
    persist_directory=directory # persists the vectors to the file system
    )
vector_index.persist()
print('count: ', vector_index._collection.count())

count:  18


In [61]:
retriever = vector_index.as_retriever(
    search_type="mmr", # Cosine Similarity
    search_kwargs={
        "k": 5, # Select top k search results
    } 
)
# retriever.get_relevant_documents("상호가 뭐지?")

In [62]:
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"


llm = HuggingFaceEndpoint(
    repo_id=repo_id, 
    # max_new_tokens=256,  
    temperature=0.1, 
    callbacks=[StreamingStdOutCallbackHandler()], 
    streaming=True,  
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True
)
# qa_chain.invoke('이전 본점 위치들 말해줘')

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\shOh\.cache\huggingface\token
Login successful


In [63]:
# # 정보 삭제
# ids = vector_index.get(0)['ids']

# print('before: ', vector_index._collection.count())
# for i in ids:
#     vector_index._collection.delete(ids=i)
# print('after :', vector_index._collection.count())

In [64]:
# 만들어놓은 벡터 retriever와 document_chain 연동

prompt = ChatPromptTemplate.from_template("""
해당 질문에 대해서 오직 주어진 context만을 기반하여 한국어로 답변을 작성해줘:
<context>
{context}
</context>

Question: {input}""")

retriever = vector_index.as_retriever(
    search_type="mmr", # Cosine Similarity
    search_kwargs={
        "k": 5, # Select top k search results
    } 
)
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# retrieval chain에 질문 넣어서 답변 생성
response = retrieval_chain.invoke({"input": "본점이 어디지?"})
# print(response["answer"])

Number of requested results 20 is greater than number of elements in index 18, updating n_results = 18




Answer: 본점은 서울특별시 강남구 테헤란로 302, 11층 100호(역삼동, 위워크타워)입니다.</s>

In [65]:
chat_history = []

In [83]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "아래의 문맥과 지금까지 나눴던 대화를 바탕으로 한국어로 사용자의 질문에 답변:\\n\\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retriever_chain = create_history_aware_retriever(llm, retriever, prompt) #retriever이랑 chat history 기반 

# 대화형으로 답변 생성
query = "발행주식총수가 몇개지"
result = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": query
    })
chat_history.append((HumanMessage(content=query), AIMessage(content=result['answer'])))

Number of requested results 20 is greater than number of elements in index 18, updating n_results = 18


?

Answer: 현재 발행주식의 총수는 176,890 주입니다.</s>

In [ ]:
chat_history